In [1]:
#apikey = X8vGMLWCXtOxkRhrgABZGj3baiaKApAn
#secret = VG1FpQufkmHA1F3n

#https://towardsdatascience.com/collecting-data-from-the-new-york-times-over-any-period-of-time-3e365504004
#https://developer.nytimes.com/docs/articlesearch-product/1/overview

In [1]:
import os
import json
import time
import requests
import datetime
import dateutil
import pandas as pd
from dateutil.relativedelta import relativedelta

### Getting data month by month (10 at a time) 

In [3]:
from dateutil.relativedelta import relativedelta
import pandas as pd
import datetime

end = datetime.date.today() + relativedelta(months=1)  # add one month to the end date
start = end - relativedelta(months=24)
months_in_range = [x.split(' ') for x in pd.date_range(start, end, freq='MS').strftime("%Y%m%d").tolist()]

months_in_range

[['20210701'],
 ['20210801'],
 ['20210901'],
 ['20211001'],
 ['20211101'],
 ['20211201'],
 ['20220101'],
 ['20220201'],
 ['20220301'],
 ['20220401'],
 ['20220501'],
 ['20220601'],
 ['20220701'],
 ['20220801'],
 ['20220901'],
 ['20221001'],
 ['20221101'],
 ['20221201'],
 ['20230101'],
 ['20230201'],
 ['20230301'],
 ['20230401'],
 ['20230501'],
 ['20230601']]

### Getting Data by Pages (in the span of 24 months) 

In [4]:
def send_request(begin_date, end_date, page):
    '''Sends a request to the NYT Archive API for given date and page.'''
    
    url = 'https://api.nytimes.com/svc/search/v2/articlesearch.json'
    api_key = 'X8vGMLWCXtOxkRhrgABZGj3baiaKApAn'
    query = 'tesla'
    sort = 'best'

    # Send the request to the API
    response = requests.get(url, params={'api-key': api_key, 'q': query, 'begin_date': begin_date, 'end_date': end_date, 'sort': sort, 'page': page})
    time.sleep(15)
    return response




def parse_response(response, start, end):
    '''Parses and returns response as pandas data frame.'''
    data = {'headline': [],  
        'date': [], 
        'doc_type': [],
        'material_type': [],
        'section': [],
        'keywords': []}
    
    articles = response.json()['response']['docs'] 
    for article in articles: # For each article, make sure it falls within our date range
        
        date = dateutil.parser.parse(article['pub_date']).date()
        data['date'].append(date)
        data['headline'].append(article['headline']['main']) 
        if 'section' in article:
            data['section'].append(article['section_name'])
        else:
            data['section'].append(None)
        data['doc_type'].append(article['document_type'])
        if 'type_of_material' in article: 
            data['material_type'].append(article['type_of_material'])
        else:
            data['material_type'].append(None)
        keywords = [keyword['value'] for keyword in article['keywords'] if keyword['name'] == 'subject']
        data['keywords'].append(keywords)
    return pd.DataFrame(data) 


def get_data(start, end):
    '''Sends and parses request/response to/from NYT Archive API for given date range.'''
    total = 0
    print('Date range: ' + str(start) + ' to ' + str(end))
    if not os.path.exists('headlines'):
        os.mkdir('headlines_pages')
    page = 5
    while True:
        response = send_request(start, end, page)
        if response.status_code != 200:
            print('Error:', response.json()['fault']['faultstring'])
            break
        df = parse_response(response, start, end)
        if len(df) == 0:
            break
        total += len(df)
        df.to_csv('headlines_pages/' + start + '-' + end + '_page' + str(page) + '.csv', index=False)
        print('Saving headlines/' + start + '-' + end + '_page' + str(page) + '.csv...')
        page += 1
    print('Number of articles collected: ' + str(total))


In [5]:
get_data(months_in_range[0][0],months_in_range[-1][0])


Date range: 20210701 to 20230601
Saving headlines/20210701-20230601_page5.csv...
Saving headlines/20210701-20230601_page6.csv...
Saving headlines/20210701-20230601_page7.csv...
Saving headlines/20210701-20230601_page8.csv...
Saving headlines/20210701-20230601_page9.csv...
Saving headlines/20210701-20230601_page10.csv...
Saving headlines/20210701-20230601_page11.csv...
Saving headlines/20210701-20230601_page12.csv...
Saving headlines/20210701-20230601_page13.csv...
Saving headlines/20210701-20230601_page14.csv...
Saving headlines/20210701-20230601_page15.csv...
Saving headlines/20210701-20230601_page16.csv...
Saving headlines/20210701-20230601_page17.csv...
Saving headlines/20210701-20230601_page18.csv...
Saving headlines/20210701-20230601_page19.csv...
Saving headlines/20210701-20230601_page20.csv...
Saving headlines/20210701-20230601_page21.csv...
Saving headlines/20210701-20230601_page22.csv...
Saving headlines/20210701-20230601_page23.csv...
Saving headlines/20210701-20230601_page24